<a href="https://colab.research.google.com/github/Carmen-GL/repo_master_/blob/main/02_Pytorch_Regresion_Lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![IDAL](https://i.imgur.com/tIKXIG1.jpg)  

#**Máster en Inteligencia Artificial Avanzada y Aplicada:  IA^3**
---


#<center>**Regresion Lineal y descenso de gradiente con PyTorch**</center>

Este notebook cubre los siguientes aspectos:

- Regresion lineal y ajuste empleando descenso de gradiente
- Implementación de modelos de este tipo empleando tensores Pytorch
- Entrenamiento del modelo de regresion lineal usando el descenso de gradiente
- Implementación de ambos empleando las clases y métodos específicamente preparados en Pytorch

## Regresión lineal

En este notebook vamos a repasar una de las técnicas básicas y fundacionales del aprendizaje máquina y de las redes neuronales: la *regresión lineal*. 
Vamos a crear un modelo que prediga la cosecha a recoger de manzanas y naranjas (*target variables*) a partir de las observaciones de temperatura, lluvia y humedad (*input variables o features*) en una región. Estos son los datos de entrenamiento:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

En un modelo de regresión lineal cada variable dependiente o *target* es estimada como la suma ponderada de las variables de entrada, más un valor constante de ajuste, conocido como *bias* :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visualmente esto significa que la cosecha de manzanas es una función lineal *o planar*  de la temperatura, la lluvía y la humedad:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

La parte de aprendizaje en una función de regresión lineal consiste en obtener un conjunto de pesos o coeficientes `w11, w12,... w23, b1 y b2` empleando los datos de entrenamiento, con la finalidad de poder hacer predicciones para nuevos datos. 
Los pesos *aprendidos* serán empleados para predecir los valores de cosecha de manzanas y naranjas en una región empleando los datos de temperatura, lluvia y humedad de esa región. . 

El entrenamiento que vamos a realizar consiste en ir ajustando los pesos ligeramente muchas veces para ir obteniendo mejores predicciones a partir de los valores resultantes conocidos y empleando una técnica de optimización ampliamente usada y conocida llamada *descenso de gradiente*.

Empezamos por importar Numpy y Pytorch.

In [ ]:
import numpy as np
import torch

## Datos de entrenamiento

Vamos a emplear para representar los datos de entrenamiento dos matrices: `inputs` y `targets`, cada fila será una observación y cada coluumna una variable
.

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [ ]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

Separamos las entradas y los targets porque hemos de trabajar separadamente con cada una de ellas. Por otro lado, se han creado como arrays Numpy porque es la forma habitual en que los vamos a encontrar: importación de los datos CSV como arrays, prepararlos y finalmente convertirlos a tensores de Pytorch.

Los convertimos a tensores PyTorch.

In [ ]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Modelo de regresion lineal desde cero

Los pesos y ajustes (`w11, w12,... w23, b1 y b2`) son representados como matrices, que en un primer momento contienen valores iniciales aleatorios. 
can also be represented as matrices, initialized as random values. 
La primera fila de `w` y el primer elemento de `b` son los coeficientes necesarios para calcular la primera variable, i.e., la cosecha de manzanas, y de forma similar la segunda para las naranjas.

In [ ]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True) #matriz de pesos, grad automático activado
b = torch.randn(2, requires_grad=True) #matriz de biases/constantes
print(w)
print(b)

tensor([[ 0.7420,  0.6785, -0.1874],
        [-1.4608, -0.3993,  1.2399]], requires_grad=True)
tensor([-0.3694, -0.9027], requires_grad=True)


`torch.randn` crea un tensor con las dimensiones dadas con elementos tomados de forma aleatoria de una [distribución normal](https://en.wikipedia.org/wiki/Normal_distribution) con media 0 y desviación estandard 1.

Por tanto, nuestro *modelo* es real y simplemente una función que realiza una multiplicación de matrices entre las entradas `inputs` y los pesos `w` (transpuestos) y añade el factor bias `b` (replicado para cada observación, diferente para cada target).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

Empelando las operaciones disponibles en Pytorch, podemos definir el modelo como sigue:

In [ ]:
def model(x):
    return x @ w.t() + b

`@` representa la multuiplicación de matrices en PyTorch, y el método `.t` devuelve un tensor transpuesto.

La matriz obtenida al emplear los datos de entrada con los coeficientes del modelo es un conjunto de predicciones para las variables objetivo *targets*.

In [ ]:
# Genero predicciones (iniciales)
preds = model(inputs)
print(preds)

tensor([[  91.1971,  -80.9740],
        [ 114.8664,  -89.6143],
        [ 144.2334, -109.5771],
        [  97.5544, -121.1936],
        [ 102.8469,  -53.2318]], grad_fn=<AddBackward0>)


In [ ]:
# Comparamos con los targets reales
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Podemos ver una gran diferencia entre las predicciones obtenidas inicialmente y los valores reales. Esto ocurre porque los pesos y bias empleados han sido inicializados aleatoriamente y no podemos esperar que esos valores se correspondan y funcionen bien directamente. Un modelo inicializado aleatoriamente no está preparado para funcionar. Se trata simplemente de un inicio.


## Función de pérdida o de coste

Antes de mejorar el modelo, necesitamos evaluar como de bien está funcionando. Para ello comparamos los resultados obtenidos con los resultados que se deberían obtener. En este caso vamos a emplear como función evladora el error cuadrático medio o **mean squared error** (MSE). 
Desglosando el pseeudo código para calcularlo sería algo así:
* Calculamos la diferencia entre las dos matrices(`preds` y `targets`).
* Elevamos al cuadrado todos los elementos y de esa forma evitamos valores negativos. 
* Calculamos la media de los elementos en una matriz resultante.

El resultado de este cálculo es un único número (MSE).

In [ ]:
# MSE 
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum` retorna la suma de todos los elementos de un tensor. El método `.numel` retorna el número de elementos de un tensor. 

Calculamos el MSE para las predicciones que hemos obtenidos con nuestro modelo. 

In [ ]:
loss = mse(preds, targets)
print(loss)

tensor(18139.0586, grad_fn=<DivBackward0>)


Así es como podemos interpretar el resultado: *de media, cada elemento en la predicción difiere de su valor correcto la raíz cuadrada del error obtenido*.
Y eso es bastante malo, dado que los resultados que pretendemos predecir están en un rango de entre 50-200. El resultado se suele llamar *loss* porque implica la pérdida que introduce el modelo entre lo que obtenemos y lo que deberíamos obtener. Cuanto **menor es la pérdida** o error, **mejor es el modelo**. 

In [ ]:
print(torch.sqrt(loss))

tensor(134.6813, grad_fn=<SqrtBackward0>)


## Cálculo de gradientes

Con Pytorch podemos calcular automáticamente los gradientes o derivadas del error con respecto de los pesos y del bias, porque hemos definido el parámetro `requires_grad` a `True`. Vamos a ver lo útil que resulta esta funcionalidad.

In [ ]:
# Calculamos gradientes. Se genera automáticamente el atributo grad de cada variable de la función de pérdida
loss.backward()

Los gradientes están ahora guardados en la propiedad `.grad` de cada tensor. Nótese que la derivada del error w.r.t. a una matriz de pesos es una matriz también de las mismas dimensiones.

In [ ]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.7420,  0.6785, -0.1874],
        [-1.4608, -0.3993,  1.2399]], requires_grad=True)
tensor([[  3108.5039,   2390.7727,   1585.8516],
        [-15498.1895, -16546.2441, -10134.0117]])


## Ajuste de pesos y bias para reducir el error

La pérdida es una [función cuadrática](https://en.wikipedia.org/wiki/Quadratic_function) de nuestros pesos y biases,  y nuestro objetivo es encontrar el conjunto de pesos donde la pérdida sea la más baja. Si dibujamos un gráfico de la pérdida con respecto a cualquiera de los pesos y bias, tendría el aspecto de la figura que se muestra abajo. Un importante detalle sobre cálculo es que el gradiente indica precisamente el ratio de cambio de la pérdida, es decir, la [pendiente](https://en.wikipedia.org/wiki/Slope) con respecto a los pesos y biases.

Si el gradiente es **positivo**

* **incrementar** el peso de esa variable ligeramente **incrementará** el error (*loss*) 
* **reducir** el peso ligeramente **reducirá** el error.

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

Si el gradiente es **negativo**:

* **incrementar** el peso de esa variable ligeramente **reducirá** el error (*loss*) 
* **reducir** el peso ligeramente **aumentará** el error.

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

El incremento o reducción del error cambiando el peso de un elemento es proporcional al gradiente del error con respecto a dicho elemento (*variable*). 
Esta observación supone la base del algoritmo de optimización por *descenso de gradiente* que será el que emplearemos para mejorar nuestro modelo (por  _descenso_ a lo largo del _gradiente_).

Dadas las relaciones descritas entre incremento/reducción del peso e incremento reducción del error, la forma en que podemos reducirlo consistirá en **restar** al peso de cada variable una pequeña cantidad proporcional al gradiente con respecto a dicha variable. 

In [ ]:
w
w.grad

tensor([[  3108.5039,   2390.7727,   1585.8516],
        [-15498.1895, -16546.2441, -10134.0117]])

In [ ]:
# Actualizamos los pesos y bias
with torch.no_grad(): #le decimos que no calcule aquí el grad porque no hace falta. Solo hace falta al calcular la pérdida
    w -= w.grad * 1e-5 # w = w - w.grad*1e-5 El valor que había menos el nuevo valor (por grad multiplicado por el learning rate 1 elevado a menos 5)
    b -= b.grad * 1e-5

Como se puede observar, hemos multiplicado los gradientes obteenidos por un número muy pequeño (`10^-5` en este caso). Esto es para asegurar que no modificamos los pesos en cantidades muy grandes. Queremos ir tomando pequeños pasos hacia la dirección de descenso siempre, no un gran salto que nos pueda desviar. Este número es lo que llamamos el ratio de aprendizaje *learning rate* del algoritmo.

Podemos usar `torch.no_grad` para indicar a PyTorch que no queremos que internamente vaya calculando gradientescuando actualizamos los pesos, ya que no es necesario y únicamente provocamos más carga computacional. 

In [ ]:
# Verificamos que la pérdida se ha reducido (probablemente muy poco)
loss = mse(preds, targets)
print(loss)

tensor(18139.0586, grad_fn=<DivBackward0>)


A continuación, será necesario resetear los gradientes a cero empleando el   método `.zero_()`. Necesitamos hacer esto porque PyTorch acumula los gradientes. Si no lo hacemos, la próxima vez que invocamos el método `.backward` en la función de pérdida, los nuevos valores de gradientes se sumarían a los existentes, lo que llevaría a resultados incorrectos e inesperados.

In [ ]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Entrenamiento del modelo usando descenso de gradiente

Como hemos visto, podemos reducir la pérdida y mejorar nuestro modelo empleando la técnica de descenso de gradiente. Así pues, podemos _entrenar_ el modelo siguiendo los siguientes pasos: 

1. Generamos predicciones

2. Calculamos el error/pérdida

3. Calculamos los gradientes c.r.a. los pesos y biases

4. Ajustamos los pesos restando una pequeña cantidad proporcianal a los gradientes obtenidos

5. Reseteamos los gradientes a cero para repetir la operación

Vamos a implementar esto paso a paso.

In [ ]:
# Generamos predicciones
preds = model(inputs)
print(preds)

tensor([[ 82.0905, -27.4557],
        [102.9706, -19.3110],
        [130.5770, -26.5073],
        [ 87.9828, -67.8447],
        [ 91.7461,  14.1158]], grad_fn=<AddBackward0>)


In [ ]:
# Calculamos el error/pérdida
loss = mse(preds, targets)
print(loss)

tensor(7718.5869, grad_fn=<DivBackward0>)


In [ ]:
# Calculamos los gradientes
loss.backward()
print(w.grad)
print(b.grad)

tensor([[  2172.9717,   1397.9363,    970.6132],
        [ -9974.1738, -10613.6172,  -6472.6147]])
tensor([  22.8734, -117.4006])


In [ ]:
# Ajustamos los pesos
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

Veamos los pesos y biases.

In [ ]:
print(w)
print(b)

tensor([[ 0.6581,  0.6167, -0.2288],
        [-1.0511,  0.0378,  1.5073]], requires_grad=True)
tensor([-0.3703, -0.8978], requires_grad=True)


Con los nuevos pesos y biases, el modelo tiene un error menor.

In [ ]:
# Calculamos la pérdida
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(5335.9121, grad_fn=<DivBackward0>)


Podemos observar que tenemos una mejora significativa, ajustando los pesos y biases segun esta técnica.

## Entrenar durante múltiples ciclos/épocas (epochs)

Para reducir el error aún más, podemos repetir el proceso de ajustar pesos y biases usando los gradientes múltiples veces. Cada iteración se llama ciclo o época (_epoch_). Vamos a entrenar el modelo 100 epochs.

In [ ]:
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [ ]:
# Calculamos el nuevo error
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(167.4839, grad_fn=<DivBackward0>)


In [ ]:
print(torch.sqrt(loss))

tensor(12.9416, grad_fn=<SqrtBackward0>)



El error ahora es bastante menor que en el momento inicial. Vamos a ver las nuevas predicciones y compremoslas con los valores target.

In [ ]:
# Prediciones
preds

tensor([[ 62.0715,  68.9378],
        [ 79.4279, 106.4170],
        [117.1245, 122.0901],
        [ 49.3473,  30.5857],
        [ 80.5174, 132.7114]], grad_fn=<AddBackward0>)

In [ ]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Observamos que las predicciones son ahora bastante cercanas a los objetivos. Podemos mejorar estos resultados entrenando más ciclos. 

## Regresión lineal empleando las funciones propias de Pytorch

Hasta ahora hemos implementado la regresión lineal y el descenso de gradiente empleando operaciones básicas sobre tensores. Sin embargo, dado que estas operaciones son un patrón común en deep learning, Pytorch provee una serie de **funciones y clases propias** específicamente preparadas para facilitar la creación y entrenamiento con tan solo unas cuantas líneas de código. 

Empezaremos importando el paqete `torch.nn` de PyTorch, el cual contiene las clases de utilidad para construir redes neuronales (_neural networks_).

In [ ]:
import torch.nn as nn

Igual que antes, vamos a representar las entradas y salidas como matrices.

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [ ]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

Vamos a usar 15 observaciones de entrenamiento para ilustrar como trabajar con conjuntos más grandes en pequeños lotes (_batches_). 

## Dataset y DataLoader

Vamos a crear un `TensorDataset`, el cual va a permitir acceder a las filas de `inputs` y sus respectivos `targets` como tuplas, además de proveer APIs estandard para trabajar con muchos diferentes datasets en PyTorch.

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

`TensorDataset` nos permite acceder a una pequeña sección de los datos de entrenamiento usando la notación de índices de array (`[0:3]` en el código anterior). Devuelve una tupla con dos elementos. El primer elemento contiene las variables de entrada de las filas seleccionadas y el segundo contiene los objetivos (`target`).

Vamos a crear también un  `DataLoader`, el cual irá dividiendo los datos en lotes (_batches_) de un tamaño predefinido mientras hace el entrenamiento. tambien aporta otras utilidades como el barajeo (_shuffling_) y el muestreo aleatorio de los datos.

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
# Definimos el data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

Ahora podemos usar el data loader en un bucle `for`. Veamos un ejemplo:

In [ ]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  87.,  65.],
        [ 87., 134.,  58.],
        [ 88., 134.,  59.],
        [ 92.,  87.,  64.],
        [ 68.,  96.,  71.]])
tensor([[ 80., 102.],
        [119., 133.],
        [118., 132.],
        [ 82., 100.],
        [104., 118.]])


En cada iteración, el data loader devuelve un lote de datos con el tamaño indicado. Si `shuffle` es `True`, "barajará" los datos antes de crear los lotes. Esto ayuda a alatorizar las entradas al algoritmo de optimización, lo cual redunda en una reducción del error más rápida.

## nn.Linear

En lugar de inicializar los pesos y biases manualmente, podemos definir el modelo usando la clase `nn.Linear` de PyTorch, la cual lo hace automáticamente.

(Más sobre `nn.linear` [aquí](https://pytorch.org/docs/stable/nn.html#linear-layers))


In [ ]:
# Define modelo.  Es equivalente a la función model que hemos creado arriba.
model = nn.Linear(3, 2) #3 son los pesos y 2 son los biases
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3048,  0.2942,  0.4785],
        [-0.0646, -0.2917,  0.1269]], requires_grad=True)
Parameter containing:
tensor([-0.2470, -0.0135], requires_grad=True)


**NOTA IMPORTANTE:** 
*La clase "nn.Linear" empleada no se refiere directamente a una regresion lineal. En realidad lo que describe es un tipo de red neuronal empleado. En concreto este es un modelo en el que las capas estan totalmente conectadas unas con otras, denominadas "fully connected" o "dense" (de forma similar a Keras). Más adelante veremos que los modelos de redes neuronales pueden estar compuestos de capas de este tipo, de capas convolucionales y otros tipos.*

Los modelos en PyTorch tienen un método muy útil llamado `.parameters`, el cual retorna una lista conteniendo todas las matrices de pesos y biases presentes en ese modelo. Para nuestro modelo de regresion lineal, tenemos una matriz de pesos y otra de biases.

In [ ]:
# Parametros
list(model.parameters())

[Parameter containing:
 tensor([[-0.3048,  0.2942,  0.4785],
         [-0.0646, -0.2917,  0.1269]], requires_grad=True),
 Parameter containing:
 tensor([-0.2470, -0.0135], requires_grad=True)]

Ahora podemos usar el modelo generado para realizar nuestras predicciones.

In [ ]:
# Genera prediciones
preds = model(inputs)
preds

tensor([[ 17.7876, -18.8161],
        [ 28.5269, -23.4394],
        [ 40.4062, -37.3631],
        [ -0.9813, -14.4484],
        [ 40.4559, -23.5911],
        [ 17.1886, -18.5889],
        [ 28.7113, -23.0207],
        [ 40.5799, -37.3008],
        [ -0.3824, -14.6755],
        [ 41.2392, -23.3996],
        [ 17.9719, -18.3974],
        [ 27.9280, -23.2122],
        [ 40.2219, -37.7818],
        [ -1.7645, -14.6399],
        [ 41.0549, -23.8183]], grad_fn=<AddmmBackward0>)

## Funcion de error o pérdida 

En lugar de definir una función de error manualmente, podemos usar la función propia de Pytorch `mse_loss`.


In [ ]:
# Importa nn.functional
import torch.nn.functional as F

El paquete `nn.functional` contiene muchas otras y útiles funciones de cálculo de error entre otras utilidades. 

(Más sobre `nn.functional`[aquí](https://pytorch.org/docs/stable/nn.functional.html))

In [ ]:
# Definimos la función de loss
loss_fn = F.mse_loss

Calculamos el error para las predicciones de nuestro modelo de la siguiente forma: 

In [ ]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(9000.4375, grad_fn=<MseLossBackward0>)


## Optimización

En lugar de manualmente manipular los pesos y biases del modelo a través de los gradientes, podemos usar el optimizador propio `optim.SGD`. SGD es la abreviatura de "stochastic gradient descent". El término estocástico indica que las muestras son seleccionadas en lotes aleatorios en lugar de individualmente.

In [ ]:
# Define optimizador
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Obsérvese que los parámetros del modelo son pasados como argumento a `optim.SGD` de forma que el optimizador sepa qué matrices de pesos y biases osn las que tiene que ir modificando durante el proceso de actualización. También podemos especificar el ratio de aprendizaje "lr" que controla la cantidad con que los parametros son modificados.

## Entrenamiento del modelo

Vamos ahora a realizar el entrenamiento del modelo. Seguiremos el mismo proceso que ya hemos visto para implementar el descenso de gradiente:

1. Generamos predicciones

2. Calculamos el error/pérdida

3. Calculamos los gradientes c.r.a. los pesos y biases

4. Ajustamos los pesos restando una pequeña cantidad proporcianal a los gradientes obtenidos

5. Reseteamos los gradientes a cero para repetir la operación

El único cambio es que ahora vamos a trabajar con lotes de datos en lugar de emplear el dataset de entrenamiento completo en cada iteracion. 

Vamos a definir una función `fit` que realizará el entrenamiento de esa forma para un número dado de ciclos (_epochs_).

In [ ]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repetir para el número especificado de epochs
    for epoch in range(num_epochs):
        
        # Entrena por lotes de datos. Xb los datos, yb las predicciones
        for xb,yb in train_dl:
            
            # 1. Generamos predicciones
            pred = model(xb)
            
            # 2. Calculamos el error/pérdida
            loss = loss_fn(pred, yb)
            
            # 3. Calculamos los gradientes
            loss.backward()
            
            # 4. Actualizamos los parámetros
            opt.step()
            
            # 5. Reseteamos los gradientes a cero
            opt.zero_grad()
        
        # Imprimimos el progreso
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Algunso detalles sobre la función definida: 

* Usamos el data loader definido previamente para obtener lotes de datos para cada iteración.

* En lugar de actualizar los parametros (pesos y biases) manualmente, usamos `opt.step` para realiarlo y `opt.zero_grad` para resetear a cer los gradientes. 

* Se ha añadido unas líneas de log que imprimen el error del último lote entrenado cada 10 iteraciones, de forma que podemos seguir en progreso de entrenamiento. `loss.item` retorna el valor actual almacenado en el tensor de error _loss_.

Vamos a entrenar 100 veces:

In [ ]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 138.8557
Epoch [20/100], Loss: 100.8392
Epoch [30/100], Loss: 183.3832
Epoch [40/100], Loss: 25.9495
Epoch [50/100], Loss: 135.4082
Epoch [60/100], Loss: 38.9266
Epoch [70/100], Loss: 20.6251
Epoch [80/100], Loss: 22.5133
Epoch [90/100], Loss: 7.5282
Epoch [100/100], Loss: 12.2580


Generamos predicciones y observemos si se acercan a los objetivos

In [ ]:

preds = model(inputs)
preds

tensor([[ 57.4033,  71.6262],
        [ 82.2962,  99.5772],
        [116.7389, 132.9645],
        [ 23.7836,  44.7695],
        [100.5494, 112.7354],
        [ 56.2455,  70.6960],
        [ 82.1688,  99.4241],
        [117.0673, 133.4781],
        [ 24.9415,  45.6998],
        [101.5798, 113.5125],
        [ 57.2759,  71.4731],
        [ 81.1384,  98.6470],
        [116.8663, 133.1176],
        [ 22.7533,  43.9924],
        [101.7072, 113.6657]], grad_fn=<AddmmBackward0>)

In [ ]:

targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Ahora las predicciones son bastante cercanas a nuestros valores objetivos. Hemos entrenado un modelo razonablemente bueno que nos permite predecir la cosecha a partir de tres variables temperatura, lluvias y humedad en una región. Podemos emplear el modelo para realizar predicciones para otra región pasandole un lote con una sola fila como entrada: 

In [ ]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.1533, 68.8140]], grad_fn=<AddmmBackward0>)

El modelo predice una cosecha de aprox. 54 tons/hect. de manzanas, y 68 tons/hect. de naranjas.

## Fin del Notebook

Referencias y modelos empleados para el Notebook: 

*   Documentación de [Pytorch](https://pytorch.org/docs/stable/index.html) 
*   [PyTorch Tutorial for Deep Learning Researchers](https://github.com/yunjey/pytorch-tutorial) by Yunjey Choi
*   [FastAI](https://www.fast.ai/) development notebooks by Jeremy Howard.
*   Documentación y cursos en [Pierian Data](https://www.pieriandata.com/)
*   Tutoriales y notebooks del curso "Deep Learning with Pytorch: Zero to GANs" de [Aakash N S](https://jovian.ai/aakashns)



